## Flow study

In [1]:
import networkx as nx
import numpy as np
import time
# Open adjencency list file and build the undirected graph
f=open("../lightningAdjList.txt", 'rb')
G=nx.read_multiline_adjlist(f, create_using=nx.Graph)
f.close()

print("Number of nodes: " + str(G.number_of_nodes()))
print("Number of edges: " + str(G.number_of_edges()))

# Read alias file and create a pub_key -> alias dic

###############################################################
#        WRITE FILE WITH INDEXES INSTEAD OF ADDRESSES
###############################################################


file_name = "demo_matrix_integer_nodes.txt"


f = open(file_name, 'w')

nodes = list(G.nodes())

capacities = nx.get_edge_attributes(G, 'capacity')

for n in nodes:
    f.write(str(nodes.index(n)) + " ")
    
    
    number_neigh = 0
    for neigh in G.neighbors(nodes[nodes.index(n)]):
        number_neigh += 1
    f.write(str(number_neigh) + "\n")
    
    
    for neigh in G.neighbors(nodes[nodes.index(n)]):
        f.write(str(nodes.index(neigh)) + " ")
        for key in capacities.keys():
            if(n in key and neigh in key):
                f.write("{\'capacity\':" + str(capacities[key]) + "}\n")
                break
f.close()

###############################################################
#        READ MATRIX WITH NODE TYPE INT
###############################################################

f=open("demo_matrix_integer_nodes.txt", 'rb')
G_int =nx.read_multiline_adjlist(f, create_using=nx.Graph, nodetype=int)
f.close()

print("Number of nodes: " + str(G_int.number_of_nodes()))
print("Number of edges: " + str(G_int.number_of_edges()))



Number of nodes: 1647
Number of edges: 8508
Number of nodes: 1647
Number of edges: 8508


In [3]:
# Initialize the adjancency matrix
def init_adjacency(G):
    A = np.zeros((G.number_of_nodes(), G.number_of_nodes()))

    nodes = list(G.nodes())

    # Add all the edges to the adjancency matrix
    for n in nodes:
        for neigh in G.neighbors(n):
            A[nodes.index(n)][nodes.index(neigh)] = 1
    return A

# Estimate the number of paths starting at one arbitrary node
def estimate_number_paths_node(G, node, iterations=1000):
    dist = {}

    # Get a list of nodes
    nodes = list(G.nodes())
    
    # Get our last vertex
    n = nodes[-1]
    
    # Get our adj matrix reset mask
    #reset_A = init_adjacency(G)
    
    for it in range(iterations):
        
        # Get a clean adj matrix
        #A = np.zeros((G.number_of_nodes(), G.number_of_nodes())) + reset_A
        
        x_t = node
        c = node
        g = 1
        t = 1

        # Ensure path will not return to the first node
        for i in range(len(A[x_t])):
            A[i][node] = 0
        
        # Init the path starting from the first node
        #######DOUBT: IS IT WORTH BUILDING A PATH LIST?##########
        path = [x_t]
        
        
        while(1):
            # Build the neighbour list
            V = []
            for neigh in G.neighbors(nodes[x_t]):
                if(not A[x_t][nodes.index(neigh)]):
                    continue
                V += [nodes.index(neigh)]

            if(len(V) == 0):
                break
            
            x_t1 = np.random.choice(V)

            x_t = x_t1
            c = x_t1
            path += [x_t]
            g = g/len(V)
            
            # Fix adj matrix so we don't come back to this node
            for i in range(len(A[x_t1])):
                A[i][x_t] = 0

            # If we got to the last node of the set we break
            if(c == n):
                break
        
        

        dist[tuple(path)] = g

        
    return dist

In [4]:
import time

t = time.time()

estimate_number_paths_node(G, 0, iterations=500)


print("Execution took: ", time.time() - t)

Execution took:  17.783737659454346


In [4]:
print("Total time estimation is: ", G.number_of_nodes() * 27, " seconds")

Total time estimation is:  44469  seconds
